# **Quantum Computing — the soft way**
### *QPlayLearn*

First of all, we install and import all the packages we are going to need to run the code

In [22]:
# Qiskit is the open-source library for quantum computing founded by IBM
import qiskit as qk
from qiskit.quantum_info import Statevector
# These are packages for plotting
import matplotlib as mpl
import matplotlib.pyplot as plt

## **QC SANDBOX #1 - Amplitude game**

We have read so much about qubits, and even done some math, now it's time to put your fingers on the keyboard! Before playing the amplitude game, let's look at some basic commands on Qiskit. We'll work mostly on Qiskit as it's the most used library, but once you get the handle of it, you can easily switch to others such as QASM. 

Let's start by creating a quantum circuit with 1 single qubit, we can also visualise it in the same way we saw in the previous section of the course.

In [20]:
# Create a quantum circuit with 1 qubit
num_qubits = 1
qc = qk.QuantumCircuit(num_qubits)

# Apply a X gate on the qubit
qc.x(0)

# Draw a graphical representation of the circuit (using matplotlib)
print(qc)

   ┌───┐
q: ┤ X ├
   └───┘


As we said, the X gate transform the initial state into $\ket{\psi_{final}} = X \ket{\psi_{initial}}$. We should know  this point not 

In [27]:
# Compute the state of corresponding to the quantum circuit
psi = Statevector(qc)
print("|psi> = ", psi.data)

|psi> =  [0.+0.j 1.+0.j]


Apply the correct gates so that the final state has amplitude $\alpha = \frac{i}{\sqrt{2}}$ and $\beta = \frac{1}{\sqrt{2}}$.